<a href="https://colab.research.google.com/github/Koftarik/Data_Processing_and_Machine_Learning_Advanced/blob/main/Gorbuntsov_BIV206_SII_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
import pandas as pd
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling2D

In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
with zipfile.ZipFile("/content/drive/MyDrive/Colab_Notebooks/data/sii_data/lab_2/miem-hse-ais-2023-lab-02.zip","r") as zip_ref:
    zip_ref.extractall("./")

In [5]:
train_dir = "train"
valid_dir = "valid"
test_dir = "test"
df_train =  pd.read_csv('train.csv')
df_valid = pd.read_csv('valid.csv')
df_test = pd.read_csv('sample_submission.csv')

print(df_train.head(5))
print(df_train.info())

              id  target_people
0  train0001.jpg              0
1  train0002.jpg              0
2  train0003.jpg              1
3  train0004.jpg              0
4  train0005.jpg              0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9003 entries, 0 to 9002
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             9003 non-null   object
 1   target_people  9003 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 140.8+ KB
None


In [6]:
print(df_valid.head(5))
print(df_valid.info())

              id  target_people
0  valid0001.jpg              1
1  valid0002.jpg              0
2  valid0003.jpg              0
3  valid0004.jpg              0
4  valid0005.jpg              0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1536 entries, 0 to 1535
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             1536 non-null   object
 1   target_people  1536 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 24.1+ KB
None


In [7]:
df_train.target_people = df_train.target_people.astype(str)
df_valid.target_people = df_valid.target_people.astype(str)
df_test.target_people = df_test.target_people.astype(str)
print(df_train.head(5))
print(df_train.info())

              id target_people
0  train0001.jpg             0
1  train0002.jpg             0
2  train0003.jpg             1
3  train0004.jpg             0
4  train0005.jpg             0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9003 entries, 0 to 9002
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             9003 non-null   object
 1   target_people  9003 non-null   object
dtypes: object(2)
memory usage: 140.8+ KB
None


In [8]:
batch_size = 64

In [10]:
datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255,
                                                            fill_mode='nearest')

train_generator = datagen.flow_from_dataframe(df_train,directory=train_dir,x_col='id',
                                                    y_col='target_people',class_mode='binary',batch_size=batch_size,
                                                    target_size=(224,224))

val_generator = datagen.flow_from_dataframe(df_valid,directory=valid_dir,x_col='id',
                                                    y_col='target_people',class_mode='binary',batch_size=batch_size,
                                                    target_size=(224,224))

test_generator = datagen.flow_from_dataframe(df_test,directory=test_dir,x_col='id',
                                                  class_mode=None, batch_size=1, shuffle = False,
                                                  target_size=(224,224))

Found 9003 validated image filenames belonging to 2 classes.
Found 1536 validated image filenames belonging to 2 classes.
Found 4523 validated image filenames.


In [11]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),include_top=False,weights="imagenet")

for layer in base_model.layers[:-11]:
    layer.trainable=False

9406464/9406464 [==============================] - 0s 0us/step


In [12]:
model=Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

model.compile(optimizer='Adam',
  loss = keras.losses.binary_crossentropy,metrics=[tf.keras.metrics.AUC(name='auc')])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0

In [13]:
model.fit(train_generator,validation_data=val_generator,epochs = 3,verbose = 1)


Epoch 1/3
141/141 [==============================] - 66s 335ms/step - loss: 0.3622 - auc: 0.9167 - val_loss: 1.7134 - val_auc: 0.8702
Epoch 2/3
141/141 [==============================] - 50s 352ms/step - loss: 0.2661 - auc: 0.9553 - val_loss: 1.5761 - val_auc: 0.8758
Epoch 3/3
141/141 [==============================] - 46s 325ms/step - loss: 0.2188 - auc: 0.9695 - val_loss: 0.8587 - val_auc: 0.9146


In [14]:
test_generator.reset()
predict = model.predict(test_generator, steps = len(test_generator.filenames))

4523/4523 [==============================] - 43s 9ms/step


In [15]:
filenames=[x+".jpg" for x in test_generator.filenames]
results=pd.DataFrame({"id":filenames,
                      "target_people":predict[:,0]})
results['id'] = results['id'].map(lambda x: str(x)[:-4])

results.to_csv("res.csv",index=False)

In [19]:
import csv
with open('/content/res.csv', 'r') as csv1, open('/content/res(2).csv', 'r') as csv2:
        f1 = csv.reader(csv1)
        f2 = csv.reader(csv2)
        for s1, s2 in zip(f1, f2):
            if s1 != s2:
                print ('False')

        print('True')

FileNotFoundError: ignored